# Лаб1. Оптимизационные модели. Моделирование и решение.

## Задание 1

### Постановка задачи

Мы решили открыть кофейню, которая делает два вида кофе "Раф кофе" за 400 рублей и "Капучино" за 300.

Чтобы сварить 1 чашку Раф кофе необходимо: 40 гр. зерен, 140 гр. молока и 5 гр. ванильного сахара. Для того чтобы получить одну чашку капучино нужно потратить: 30 гр. зерен, 120 гр. молока На складе есть: 500 гр. зерен, 2000 гр. молока и 40 гр. ванильного сахара.

Необходимо найти план варки кофе, обеспечивающий максимальную выручку от их реализации, учитывая, что нам не важно мнение покупателей, главное потратить весь ванильный сахар.

Требуется построить математическую модель (описать ее в блокноте при помощи Markdown), создать два блокнота  в Google Colab (среда выполнения: Python, Julia). Построить модели на соответствующем языке. Решить задачу. Подготовить один слайд для презентации результата (визуализация результата) (используйте latex и Overleaf; при создании слайда в latex примените Qwen, Deepseek или др. ).

### Математеческая модель

- $x$ — кол-во чашек кофе **раф**  
- $y$ — кол-во чашек кофе **капучино**  

Ограничения:  
- $40x + 30y \le 500$ — ограничение по зёрнам (гр.)  
- $140x + 500y \le 2000$ — ограничение по молоку (гр.)  
- $5x = 40$ — ограничение по ванильному сахару (гр.)

Ограничения целостности?:
- $x \ge 0$
- $y \ge 0$
- $x \in \mathbb{Z}$
- $y \in \mathbb{Z}$

Выручка:
- $400x + 300y \to  max$

### Код

In [5]:
:dep good_lp = "1.14.0"

use good_lp::*;

let mut vars = variables!();
let x = vars.add(variable().min(0.0));
let y = vars.add(variable().min(0.0));

let problem = vars
    .maximise(400.0 * x + 300.0 * y)
    .using(default_solver)
    .with(constraint!(40.0 * x + 30.0 * y <= 500.0))
    .with(constraint!(140.0 * x + 120.0 * y <= 2000.0))
    .with(constraint!(5.0 * x == 40.0));

let solution = problem.solve()?;

println!("x = {}", solution.value(x));
println!("y = {}", solution.value(y));

let revenue = solution.eval(400.0 * x + 300.0 * y);
println!("Максимальная выручка = {}", revenue);

x = 8
y = 6
Максимальная выручка = 5000
